<a href="https://colab.research.google.com/github/Pavankumar3012/AI-Track---ML/blob/master/assignment6b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
data = pd.read_csv("kc_house_data.csv")
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [12]:
data = data[['price', 'sqft_living']]
data

,price,sqft_living
0,221900.0,1180
1,538000.0,2570
2,180000.0,770
3,604000.0,1960
4,510000.0,1680
...,...,...
21608,360000.0,1530
21609,400000.0,2310
21610,402101.0,1020
21611,400000.0,1600


In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data = scaler.fit_transform(data)

In [ ]:
data= pd.DataFrame(data=data, columns=["price","sqft_living"])
data

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data.iloc[:, 1], data.iloc[:, 0], test_size = 0.10, random_state = 42)

In [8]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(19451,) (19451,) (2162,) (2162,)


In [9]:
batch_size = 32
n_minibatches = x_train.shape[0] // batch_size

def create_mini_batches(x_tr, y_tr, batch_size):
    mini_batches = [] 
    global n_minibatches
    for i in range(n_minibatches + 1): 
        X_mini = x_tr.iloc[i * batch_size : (i + 1) * batch_size]
        Y_mini = y_tr.iloc[i * batch_size : (i + 1) * batch_size]
        mini_batches.append((X_mini, Y_mini)) 
        if x_tr.shape[0] % batch_size != 0:
            X_mini = x_tr.iloc[i * batch_size: x_tr.shape[0]]
            Y_mini  = y_tr.iloc[i * batch_size: y_tr.shape[0]]
        mini_batches.append((X_mini,Y_mini)) 
    return mini_batches 

mini_batches_all = create_mini_batches(x_train, y_train, batch_size)

In [10]:
m = 1 #Initial value of slope
c = -1 #Initial value of intercept
lr = 0.1 #Learning Rate
delta_m = 1 #Initialising Δm
delta_c = 1 #Initialising Δc
max_iters = 10 #Maximum number of iterations  
iters_count = 0 #Counting Iterations


def deriv(m_f, c_f, datax, datay):
    m_deriv = 0
    c_deriv = 0
    for i in range(datax.shape[0]):
        x, y = datax.iloc[i], datay.iloc[i]
        m_deriv += (y-m_f*x-c_f)*x
        c_deriv += (y-m_f*x-c_f)
        m_deriv = -m_deriv/len(datax)
        c_deriv = -c_deriv/len(datay)
    return m_deriv, c_deriv  


while iters_count < max_iters:
    for i in range(n_minibatches):
        x_b, y_b = mini_batches_all[i]
        delta_m, delta_c = deriv(m, c, x_b, y_b)
        delta_m = -lr * delta_m
        delta_c = -lr * delta_c
        m += delta_m
        c += delta_c
    iters_count += 1
    print(f"Iteration: {iters_count}\tValue of m: {m}, \tValue of c: {c}")

print(f"\nThe local minima occurs at: {m}, {c}")


Iteration: 1	Value of m: 0.7854519952272078, 	Value of c: -0.42623490300776085
Iteration: 2	Value of m: 0.6991958183454019, 	Value of c: -0.1998127151903294
Iteration: 3	Value of m: 0.664502091937808, 	Value of c: -0.1104670174729536
Iteration: 4	Value of m: 0.6505413559816704, 	Value of c: -0.07521396904228948
Iteration: 5	Value of m: 0.6449210012033875, 	Value of c: -0.06130526796593927
Iteration: 6	Value of m: 0.6426573033205328, 	Value of c: -0.05581818221857195
Iteration: 7	Value of m: 0.6417451379058001, 	Value of c: -0.05365366755820519
Iteration: 8	Value of m: 0.641377407070127, 	Value of c: -0.0527998970002293
Iteration: 9	Value of m: 0.6412290908908382, 	Value of c: -0.05246316698493894
Iteration: 10	Value of m: 0.6411692428991926, 	Value of c: -0.052330372341350184

The local minima occurs at: 0.6411692428991926, -0.052330372341350184


In [13]:
import numpy as np

x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [14]:
y_pred_train = []
for i in x_train:
  y_p_tr = (m * i) - c
  y_pred_train.append(y_p_tr)
y_pred_train = np.array(y_pred_train)

In [15]:
y_pred_test = []
for i in x_test:
  y_p_te = (m * i) - c
  y_pred_test.append(y_p_te)
y_pred_test = np.array(y_pred_test)

In [16]:
import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error


#Training Accuracies
mse = math.sqrt(mean_squared_error(y_train, y_pred_train)) 
print('Root mean square error', mse) 
mse = (mean_squared_error(y_train, y_pred_train)) 
print('Mean square error', mse) 
mae=mean_absolute_error(y_train, y_pred_train)
print('Mean absolute error', mae)

Root mean square error 0.7113934377666099
Mean square error 0.5060806232973954
Mean absolute error 0.4716670901963416


In [17]:
mse = math.sqrt(mean_squared_error(y_test, y_pred_test)) 
print('Root mean square error', mse) 
mse = (mean_squared_error(y_test, y_pred_test)) 
print('Mean square error', mse) 
mae=mean_absolute_error(y_test, y_pred_test)
print('Mean absolute error', mae)

Root mean square error 0.762331323793402
Mean square error 0.5811490472366007
Mean absolute error 0.49207871795139424
